In [27]:
import sys
import os

# Add the repo root (one level up from notebooks) to Python path
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)

# Now imports work
from options_pricing.black_scholes import black_scholes_price
from options_pricing.monte_carlo import monte_carlo_option_pricing
from data.data_loader import fetch_stock_data, compute_annualized_volatility, get_latest_price


import yfinance as yf

# Fetch stock data
ticker = "AAPL"
data = fetch_stock_data(ticker, start="2024-01-01", end="2025-01-01")

# Spot price & volatility as scalars
S0 = float(get_latest_price(data))
sigma = float(compute_annualized_volatility(data))

# Parameters
K = 200
T = 0.5
r = 0.05

# Price options
call_price = black_scholes_price(S0, K, T, r, sigma, option_type="call")
put_price = monte_carlo_option_pricing(S0, K, T, r, sigma,
                                       option_type="put", n_simulations=100000, seed=42)

print(f"Spot Price: {S0:.2f}, Volatility: {sigma:.2%}")
print(f"Call Price (BS): {call_price:.2f}")
print(f"Put Price (MC): {put_price:.2f}")



[*********************100%***********************]  1 of 1 completed

Spot Price: 249.53, Volatility: 22.28%
Call Price (BS): 55.35
Put Price (MC): 0.88



c:\Users\datog\OneDrive\Desktop\options-pricing-library\options-pricing-library\data\data_loader.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(stock_data['Close'].iloc[-1])
c:\Users\datog\venv\Lib\site-packages\numpy\_core\fromnumeric.py:4062: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
c:\Users\datog\OneDrive\Desktop\options-pricing-library\options-pricing-library\data\data_loader.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(sigma)


In [8]:
S0 = data['Close'].iloc[-1]  # last available closing price
